### November 10 Objective

Priority 1: Clean-up code into the python code.

Priority 2: Develop a segmentdef schema and process.

Priority 3: Digest the Network into all the lines. Display them on MAMP page.

next steps:
* Display and visualization
* How can we use D3?

#### Imports

In [12]:
import dbactions as dba
import interpretnetwork as intnet
import osmtonetwork as om
import mapping

In [13]:
# dir(dba)

## Set-up Essential

### Build Our Network.

In [14]:
# xmlfile = '/Users/Filip/Dropbox/code/QGIS/Data/Tramwaje-krakow/siec.xml'
# G2 = om.buildOSMnx(xmlfile)
# om.cachenetwork(G2,"nov10")
G = om.getcached("nov10")
len(G.nodes())

7521

In [15]:
db = dba.db
namednodes = intnet.get_namednodes(G)

## Set-up Specific

In [5]:
relation_13 = dba.get_relations(13)[0]
relation_18 = dba.get_relations(18)[0]
relation_1 = dba.get_relations(1)[0]
relation_50 = dba.get_relations(50)[0]


# Examples

In [6]:


# namednodes = intnet.get_namednodes(G)

# nodeA, nodeB, path_len, path= intnet.get_shortestpath(G, intnet.get_stop_nodes(relation_13[0], namednodes)[0], intnet.get_stop_nodes(relation_13[10], namednodes)[0])

# mapping.path(G, path)

In [7]:
# stradom = intnet.get_stop_nodes(relation_13[15], namednodes)
# len(stradom)

# wawel = intnet.get_stop_nodes(relation_13[16], namednodes)
# len(wawel)

# stopA_node, stopB_node, shortest_path_length, shortest_path = intnet.get_twostops_shortestpath(G, wawel , stradom)
# print shortest_path
# mapping.path(G, shortest_path)

### From network, get nodes that correspond to a stop

In [8]:
def relationnodes(relation):
    listoflists = []
    for stop in relation:
        listoflists.append( intnet.get_stop_nodes(stop, namednodes) )
    return listoflists

### From Network, get named nodes, first named node, first named node without the current one. Confusing

In [9]:
# len(shortest_path)
# intnet.path_remove_namednode(namednodes, shortest_path, "Wawel")
# len(shortest_path)

In [10]:
# intnet.get_stop_nodes(relation_1[12], namednodes)

In [11]:
# cmentarz_podgorski = intnet.get_stop_nodes(relation_13[10], namednodes)
# len(cmentarz_podgorski)

# wawel = intnet.get_stop_nodes(relation_13[16], namednodes)
# len(wawel)

# stopA_node, stopB_node, shortest_path_length, shortest_path = intnet.get_twostops_shortestpath(G, wawel , cmentarz_podgorski)
# print shortest_path


In [12]:
# intnet.path_get_intermediate_namednodes(namednodes, shortest_path, u'Cmentarz Podg\xf3rski', u'Wawel')

# Get Start Node - s0

In [17]:
"""
get start node

input: relation

output
start node - nx


"""


def tramnet_get_start_node(relation):

    s0 = relation[0] #  "Krowodrza Górka"
    s1 = relation[1] # "Bratysławska"


#    ASSUMES GEOLOCATED STOPS

    s0nodes = intnet.get_stop_nodes(s0, namednodes)
    s1nodes = intnet.get_stop_nodes(s1, namednodes)

    #find shortest path
    a_node, b_node, path_len, path = intnet.get_twostops_shortestpath(G, s0nodes , s1nodes )


    #save the start node
    start_node = a_node

    return start_node

In [18]:
tramnet_get_start_node(relation_13)

{'name': u'Nowy Bie\u017can\xf3w',
 'osmid': 287586332,
 'x': 20.0232538,
 'y': 50.0157031}

# Step 2: Get next node

In [19]:
"""
get next node and path

input:
start_node
relation
current_stop_number

output
next node - nx
path - list of node id's

requires:
namednodes

"""


def tramnet_get_next_node_and_path(start_node, relation, current_stop):
    
    i = current_stop
    StopAName = relation[i]
    StopBName = ""
    
    #define stop 0
    s0nodes = []
    s0nodes.append(start_node)
    
    #define stop 2
    s2nodes = []
    
#     print StopBName
    
    #FUTURE Convert this into a recurvsive function.
    if intnet.get_stop_nodes(relation[i+2], namednodes) == []:
        if intnet.get_stop_nodes(relation[i+3], namednodes) == []:
            if intnet.get_stop_nodes(relation[i+4], namednodes) == []:
                return "stop too far to work"
            else:
                StopBName = relation[i+4]
        else:
            StopBName = relation[i+3]
    else:
        StopBName = relation[i+2]
        
    s2nodes = intnet.get_stop_nodes(StopBName, namednodes)
#     print s2nodes
    
    # get the shortest path.
    a_node, c_node, path_len, path = intnet.get_twostops_shortestpath(G, s0nodes , s2nodes )

    
    a = start_node
    b_possibilities = intnet.path_get_intermediate_namednodes(namednodes, path, StopAName, StopBName)
#     print b_possibilities
    b = b_possibilities[0] #this is kinda basic. Just get the first that works.
    
    a, next_node, path_len_out, path_out = intnet.get_shortestpath(G, a, b)
        
    return next_node, path_len_out, path_out

# Step 3: get last node

In [20]:
"""

tramnet_get_last_node

input: secondtolastnode, relation

output
path

"""

def tramnet_get_last_node(secondtolastnode, relation):
    # get the exact start node (in a list of 1 element, because that's how our shortest path algo works)
    s0nodes = []
    s0nodes.append( secondtolastnode )
    
    #make a list of possible n+1 (last) nodes
    s1 = relation[len(relation)-1]
    s1nodes = intnet.get_stop_nodes(s1, namednodes)

    #find shortest path
    a_node, last_node, path_len_out, path_out = intnet.get_twostops_shortestpath(G, s0nodes , s1nodes )

    return last_node, path_len_out, path_out


    

# The Big shebang. Get the WHOLE path for a relation.

* Add a way to see what's happening

In [21]:
def get_whole_path_alt(relation):
#     global relnodes
#     relnodes = relationnodes(relation)
    
    whole_path_len = 0
    whole_path = []
    node_list = []

    #start
    start_node = tramnet_get_start_node(relation)
    node_list.append(start_node)
    
    #middle
    
    
    a_node = start_node #initialize
    
    for i in range(0, len(relation)-3):
        
#         print relation[i]

        b_node, path_len_out, path_out = tramnet_get_next_node_and_path(a_node, relation, i)

        node_list.append(b_node)
        
        whole_path_len = whole_path_len + path_len_out
        
        whole_path.extend( path_out )

        #start over with the next node in the list
        a_node = b_node
    
    #end
    last_node, path_len_out, path_out = tramnet_get_last_node(a_node, relation)
    
    whole_path_len = whole_path_len + path_len_out
    whole_path.extend( path_out )
    node_list.append(last_node)
    
    
    return whole_path, whole_path_len, node_list

In [22]:
len(relation_50)

21

In [19]:
whole_path, whole_path_len, node_list = get_whole_path_alt(dba.get_relations(24)[1])


In [20]:
mapping.path(G, whole_path)

In [29]:
print dba.get_relations(24)[0]

[u'Kurdwan\xf3w', u'Witosa', u'Nowos\u0105decka', u'Piaski Nowe', u'Dauna', u'Bie\u017canowska', u'Kabel', u'Dworcowa', u'Cmentarz Podg\xf3rski', u'Powsta\u0144c\xf3w Wielkopolskich', u'Plac Bohater\xf3w Getta', u'\u015aw.Wawrzy\u0144ca', u'Miodowa', u'Starowi\u015blna', u'Poczta G\u0142\xf3wna', u'Plac Wszystkich \u015awi\u0119tych', u'Filharmonia', u'Teatr Bagatela', u'Batorego', u'Plac Inwalid\xf3w', u'Urz\u0119dnicza', u'Biprostal', u'Uniwersytet Pedagogiczny', u'G\u0142owackiego', u'Bronowice', u'Wesele', u'Bronowice Wiadukt', u'Bronowice Ma\u0142e']


In [23]:
path = [2419720287, 2419720306, 2419720304, 4987882579, 3754628344, 2419720295, 3754628342, 2419720291, 2419720290, 3754628341, 2419720293, 4557118642]
remove_this = ""

# get_named_nodes_in_path(namednodes, path, remove_this)

In [30]:
print relation_1[28]

Salwator


In [26]:
mapping.path(whole_path)

TypeError: path() takes exactly 2 arguments (1 given)

In [31]:
for item in db.mpkrelations.find():
    print item["line_number"]

1
1
2
2
3
3
4
4
5
5
6
6
8
8
9
9
10
10
11
11
12
12
13
13
14
14
16
16
18
18
19
19
20
20
21
21
22
22
23
23
24
24
24
24
50
50
52
52
70
70
72
72


In [28]:
mapping.path(whole_path)

TypeError: path() takes exactly 2 arguments (1 given)

In [ ]:
coords = []
type ( json.dumps({'type': 'LineString', 'coordinates': coords} ) )

In [23]:
pathlist= []

for item in db.mpkrelations.find():
    print item["line_number"]
    try:
        whole_path, whole_path_len, node_list = get_whole_path_alt(item['stop_list'])
        pathlist.append(whole_path)
    except:
        False

1
1
2
2
3
3
4
4
5
5
6
6
8
8
9
9
10
10
11
11
12
12
13
13
14
14
16
16
18
18
19
19
20
20
21
21
22
22
23
23
24
24
24
24
50
50
52
52
70
70
72
72


In [24]:
mapping.pathlist(G, pathlist)

In [26]:
pathlist

[[213578451,
  1770978588,
  213578455,
  1770978584,
  1770978582,
  213578457,
  1770978576,
  1770978574,
  1770978572,
  1770978569,
  1770978567,
  1770978566,
  1770978564,
  1770978369,
  1770978369,
  1770978556,
  1770978550,
  1770978544,
  1770978541,
  1770978539,
  1770978537,
  1770978366,
  1770978366,
  4556178681,
  1770978502,
  1770978496,
  213578411,
  4556178675,
  213578409,
  4556178678,
  1770978493,
  4556178683,
  1770978489,
  213578407,
  4556178679,
  213578404,
  1770978486,
  4556178677,
  1770978485,
  2387955394,
  2387955394,
  1770978476,
  4556105016,
  1770978466,
  1770978464,
  1770978460,
  1770978458,
  1770978453,
  1770978450,
  1770978446,
  1770978444,
  1770978441,
  1770978437,
  1770978433,
  1770978430,
  213579010,
  213578767,
  1769088042,
  1769088039,
  213585832,
  777395555,
  777395555,
  1769088035,
  286858516,
  286858519,
  213585840,
  1769088018,
  213585842,
  1769088013,
  1769088006,
  213585843,
  213585845,
  17690880